# Basic

In [4]:
import numpy as np
import pandas as pd
import itertools
import math, random, time, collections, pickle
from multiprocessing import Pool
import seaborn as sns
import matplotlib.pyplot as plt
from multiprocessing import Pool
sns.set_style("whitegrid")

# MCMC

In [5]:
M = np.zeros(shape=([28, 28]))
with open("data/letter_transition_matrix.csv", "r") as f:
    lines = f.readlines()
    assert len(lines)==28, "The size of the alphabet is 28"
    for i in range(28):
        M[i, :] = [float(x) for x in lines[i].split(",")]
logM = np.log(M) # 0 exists

P = np.zeros(shape=([28]))
with open("data/letter_probabilities.csv", "r") as f:
    line = f.readline()
    assert len(lines)==28, "The size of the alphabet is 28"
    P = [float(x) for x in line[:-1].split(",")]
logP = np.log(P)

with open("data/alphabet.csv", "r") as f:
    line = f.readline()
    assert len(lines)==28, "The size of the alphabet is 28"
    Alphabet = line[:-1].split(",")
    content2idx = {}
    idx2content = {}
    for i in range(28):
        content2idx[Alphabet[i]] = i
        idx2content[i] = Alphabet[i]

with open("test_ciphertext.txt", "r") as f:
    ciphertext = f.read()[:-1] # replace \n at the end of the file
with open("test_plaintext.txt", "r") as f:
    plaintext = f.read()[:-1] # replace \n at the end of the file

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys


In [8]:
def checkvalid(content):
    transition = collections.Counter()
    for i in Alphabet:
        transition[i] = collections.Counter()
    for i in range(1, len(content), 1):  # including /n
        transition[content[i - 1]][content[i]] += 1

    notrepeat = []  # ' ' & '.'
    for a in Alphabet:
        if (transition[a][a] == 0):
            notrepeat.append(a)

    set1 = []  # " "
    set2 = []  # "."
    for a in notrepeat:
        if (len(transition[a]) == 0):
            set2.append(a)
        elif (len(transition[a]) == 1 and list(transition[a].keys())[0] in notrepeat):
            set1.append(list(transition[a].keys())[0])
            set2.append(a)

    if (len(transition) == 28):  # All the char have occurred. So we must have a ". " pair.
        return set2, set1
    else:
        return set2, notrepeat
    
class MCMC:
    def __init__(self, ciphertext, answer):
        self.alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '.']
        alphabet = self.alphabet.copy()
        self.answer = answer
        self.idx2alpha = dict(zip(range(28), self.alphabet))
        random.shuffle(alphabet)
        self.cur_f = dict(zip(alphabet, range(28)))
        self.ciphertext_transition = collections.Counter()
        for a in self.alphabet:
            self.ciphertext_transition[a] = collections.Counter()
        for i in range(1, len(ciphertext), 1): #including /n
            self.ciphertext_transition[ciphertext[i]][ciphertext[i-1]] += 1
        self.ciphertext = ciphertext
        self.set1, self.set2 = checkvalid(self.ciphertext)
        
    def Pf(self, code2idx):
        logPf = logP[code2idx[self.ciphertext[0]]]
        for a in self.alphabet:
            for b in self.alphabet:
                if self.ciphertext_transition[a][b]!=0:
                    if logM[code2idx[a], code2idx[b]] == -math.inf:
                        return "not exist"
                    else:
                        logPf += self.ciphertext_transition[a][b] * logM[code2idx[a], code2idx[b]]
        
        return logPf
    
    def generate_f(self, oldf, set1, set2):
        # set1: "."
        # set2 : " "
        a, b = random.sample(self.alphabet, 2)
        f2 = oldf.copy()
        f2[a] = self.cur_f[b]
        f2[b] = self.cur_f[a]
        for i, v in f2.items():
            if (v == 27 and i not in set1):
                candidate = random.sample(self.set1, k=1)[0]
                tmp = f2[i]
                f2[i] = f2[candidate]
                f2[candidate] = tmp
            if (v == 26 and i not in set2):
                candidate = random.sample(self.set2, k=1)[0]
                tmp = f2[i]
                f2[i] = f2[candidate]
                f2[candidate] = tmp
        return f2

    
    def decode(self):
        s = ""
        for c in self.ciphertext:
            s += self.idx2alpha[self.cur_f[c]]
        return s
    
    def accuracy(self, answer):
        s = 0
        assert len(self.ciphertext)==len(answer), "answer length not align"
        for c in range(len(self.ciphertext)):
            if (self.idx2alpha[self.cur_f[self.ciphertext[c]]]==answer[c]):
                s += 1
#         print("Accuracy {}/{}".format(s, len(self.ciphertext)))
        return s
        
    
    def run(self, n=5000, printacc=True):
        start_time = time.time()
        loglikelihood = []
        times = []
        acc = []
        accepted = []
        for t in range(n):
            f2 = self.generate_f(self.cur_f, self.set1, self.set2)
            pf2 = self.Pf(f2)
            pf1 = self.Pf(self.cur_f)
            rand = random.random()

            if pf1=="not exist" and pf2=="not exist":
                if random.random()<0.5:
                    accepted.append(True)
                    self.cur_f= f2
                else:
                    accepted.append(False)
            elif pf1=="not exist":
                accepted.append(True)
                self.cur_f = f2
            elif pf2=="not exist":
                accepted.append(False)
                pass
            
            elif pf2-pf1>5:
                self.cur_f = f2
                accepted.append(True)
            
            elif rand<min(1, np.exp(pf2-pf1)):
                self.cur_f = f2
                accepted.append(True)
                
            else:
                accepted.append(False)
                
    
            if self.Pf(self.cur_f) != "not exist":
                times.append(t)
                loglikelihood.append(self.Pf(self.cur_f))
                acc.append(100.*self.accuracy(self.answer)/len(self.ciphertext))
            else:
                times.append(math.nan)
                loglikelihood.append(math.nan)
                acc.append(math.nan)
                
            if(np.sum(accepted[-1000:])==0):
                break
                
                
            if printacc:
                if (t%1000==0):
                    print("Accuracy {}/{}, Log-Liklihood {}, time {} ({}/{})".format(self.accuracy(self.answer), len(self.answer), self.Pf(self.cur_f), time.time()-start_time, t, n))
        return times, loglikelihood, acc, accepted

In [9]:
def run(seed):
    random.seed(time.time()*random.random())
    mcmc = MCMC(ciphertext=ciphertext, answer=plaintext)
    times, loglikelihood, acc, accepted = mcmc.run()
    return loglikelihood, acc

p = Pool(processes=100)
data = p.map(run, [random.random()*10000 for x in range(1000)])
p.close()

0.0 1
0.0 1
0.0 1
0.0 1
0.0 1
0.0 1
0.0 1
0.0 1
0.0 1
0.0 1
nan 1
Accuracy 316/5287, Log-Liklihood -31259.001113980412, time 0.09000682830810547 (0/5000)
Accuracy 993/5287, Log-Liklihood -30558.30543300076, time 0.09393501281738281 (0/5000)
Accuracy 0/5287, Log-Liklihood -33116.21013512834, time 0.08151507377624512 (0/5000)
Accuracy 1086/5287, Log-Liklihood -32448.844681812556, time 0.07740902900695801 (0/5000)
Accuracy 165/5287, Log-Liklihood -30719.67268805379, time 0.07865405082702637 (0/5000)
Accuracy 993/5287, Log-Liklihood -27177.91302350653, time 0.07220005989074707 (0/5000)
Accuracy 993/5287, Log-Liklihood -26834.67537541475, time 0.06979894638061523 (0/5000)
Accuracy 90/5287, Log-Liklihood -30675.066119449144, time 0.07479119300842285 (0/5000)
Accuracy 947/5287, Log-Liklihood -31075.137442942552, time 0.06198310852050781 (0/5000)
Accuracy 993/5287, Log-Liklihood -27967.954210479173, time 0.0706632137298584 (0/5000)
Accuracy 46/5287, Log-Liklihood not exist, time 0.060438871383

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:117: RuntimeWarning: overflow encountered in exp


0.0 1
Accuracy 232/5287, Log-Liklihood -34901.919631162054, time 0.018030166625976562 (0/5000)
nan 1


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:117: RuntimeWarning: overflow encountered in exp


Accuracy 144/5287, Log-Liklihood not exist, time 0.0434727668762207 (0/5000)
nan 1
nan 1
nan 1
nan 1
nan 1
nan 1
nan 1
nan 1
nan 1
nan 1


...

Process ForkPoolWorker-188:
Process ForkPoolWorker-115:
  File "/usr/local/Cellar/python/3.7.2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/Cellar/python/3.7.2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "<ipython-input-9-e841befdd1d6>", line 4, in run
    times, loglikelihood, acc, accepted = mcmc.run()
  File "/usr/local/Cellar/python/3.7.2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "<ipython-input-8-2336979b9352>", line 125, in run
    if self.Pf(self.cur_f) != "not exist":
  File "<ipython-input-8-2336979b9352>", line 101, in run
    pf1 = self.Pf(self.cur_f)
  File "/usr/local/Cellar/python/3.7.2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/pool.py"

  File "<ipython-input-9-e841befdd1d6>", line 4, in run
    times, loglikelihood, acc, accepted = mcmc.run()
  File "<ipython-input-8-2336979b9352>", line 125, in run
    if self.Pf(self.cur_f) != "not exist":
  File "<ipython-input-9-e841befdd1d6>", line 4, in run
    times, loglikelihood, acc, accepted = mcmc.run()
  File "/usr/local/lib/python3.7/site-packages/numpy/core/fromnumeric.py", line 2076, in sum
    initial=initial)
  File "<ipython-input-9-e841befdd1d6>", line 4, in run
    times, loglikelihood, acc, accepted = mcmc.run()
KeyboardInterrupt
  File "<ipython-input-8-2336979b9352>", line 128, in run
    acc.append(100.*self.accuracy(self.answer)/len(self.ciphertext))
  File "<ipython-input-8-2336979b9352>", line 128, in run
    acc.append(100.*self.accuracy(self.answer)/len(self.ciphertext))
Traceback (most recent call last):
  File "<ipython-input-8-2336979b9352>", line 128, in run
    acc.append(100.*self.accuracy(self.answer)/len(self.ciphertext))
  File "<ipython-input-8

  File "<ipython-input-8-2336979b9352>", line 101, in run
    pf1 = self.Pf(self.cur_f)
  File "<ipython-input-8-2336979b9352>", line 51, in Pf
    logPf += self.ciphertext_transition[a][b] * logM[code2idx[a], code2idx[b]]
  File "/usr/local/lib/python3.7/site-packages/numpy/core/fromnumeric.py", line 86, in _wrapreduction
    return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
  File "<ipython-input-8-2336979b9352>", line 128, in run
    acc.append(100.*self.accuracy(self.answer)/len(self.ciphertext))
  File "<ipython-input-8-2336979b9352>", line 100, in run
    pf2 = self.Pf(f2)
  File "<ipython-input-8-2336979b9352>", line 86, in accuracy
    if (self.idx2alpha[self.cur_f[self.ciphertext[c]]]==answer[c]):
  File "<ipython-input-8-2336979b9352>", line 86, in accuracy
    if (self.idx2alpha[self.cur_f[self.ciphertext[c]]]==answer[c]):
  File "<ipython-input-8-2336979b9352>", line 86, in accuracy
    if (self.idx2alpha[self.cur_f[self.ciphertext[c]]]==answer[c]):
  File "/usr/loca

  File "/usr/local/Cellar/python/3.7.2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/Cellar/python/3.7.2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
KeyboardInterrupt
  File "<ipython-input-8-2336979b9352>", line 125, in run
    if self.Pf(self.cur_f) != "not exist":
  File "<ipython-input-9-e841befdd1d6>", line 4, in run
    times, loglikelihood, acc, accepted = mcmc.run()
KeyboardInterrupt
  File "/usr/local/Cellar/python/3.7.2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
KeyboardInterrupt
Traceback (most recent call last):
  File "<ipython-input-8-2336979b9352>", line 100, in run
    pf2 = self.Pf(f2)
Traceback (most recent call last):
  File "<ipython-input-8-2336979b9352>", line 12

  File "/usr/local/Cellar/python/3.7.2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "<ipython-input-9-e841befdd1d6>", line 4, in run
    times, loglikelihood, acc, accepted = mcmc.run()
  File "/usr/local/Cellar/python/3.7.2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.7.2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.7.2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.7.2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
KeyboardInterrupt
  File "<ipython-input-9-e841befdd1d6>", line 4, in run
    tim

In [ ]:
likelih = []
accuracy = []
for i in data:
    likelih = likelih + i[0]
    accuracy = accuracy + i[1]

likelih = [int(x/100)*100 for x in likelih if x==x]
accuracy = [x for x in accuracy if x==x]

In [ ]:
sns.lineplot(x=likelih, y=accuracy, ci= "sd", err_style = "band")
plt.xlabel("Log-Likelihood")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
fmri = sns.load_dataset("fmri")
fmri.head()

In [ ]:
ax = sns.lineplot(x=list(fmri["timepoint"]), y=list(fmri["signal"]))
# ax = sns.lineplot(x="timepoint", y="signal", hue="event", style="event", data=fmri)